In [45]:
import os
import time
import torch
import pickle

from torch import nn 
torch.set_default_tensor_type('torch.DoubleTensor')
from torch.utils.data import DataLoader
import torch.nn.functional as Functional
from torch.autograd import Variable
from sklearn import preprocessing
import numpy as np

from AE import Autoencoder 

In [17]:
input_dims = 9*5
model = Autoencoder(input_dims = 9*5, l1_size = 32, l2_size = 16, l3_size = 8)#.cuda()


dataset = np.load("Data/unnormalized_data.npy")
row,cols,num_images = dataset.shape
lobs = []
for i in range(num_images):
    lob = dataset[:,:,i]
    lob = lob.flatten()
    lobs.append(lob)
    
len_data = len(lobs)
train_len = int(0.7*len_data)





In [37]:
minmax_scale = preprocessing.MinMaxScaler().fit(lobs)
df_minmax = minmax_scale.transform(lobs)
trainloader = DataLoader(df_minmax, batch_size = 256, shuffle = True)

ValueError: Expected 2D array, got 1D array instead:
array=[981.         394.         957.         170.         414.
   1.           1.           1.           1.           1.
  70.          70.          70.          70.          70.
   1.           1.           1.           1.           1.
   0.           0.           0.           0.           0.
   0.           0.           0.           0.           0.
   0.           0.           0.           0.           0.
   0.           0.           0.           0.           0.
   1.9          1.91666667   1.93333333   1.95         1.96666667].
Reshape your data either using array.reshape(-1, 1) if your data has a single feature or array.reshape(1, -1) if it contains a single sample.

In [19]:
criterion = nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr = 1e-3, weight_decay = 1e-5)

In [20]:
epochs = 100
for epoch in range(epochs):
    running_loss = 0.0
    for data in trainloader:    
        lob = Variable(data)#.cuda()
        #===============forward==================
        output = model(lob)    
        loss = criterion(output, lob)
        #===============backward=================
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        #===============log======================
        running_loss += loss.item()
    loss = running_loss / len(trainloader)
    print('epoch [{}/{}], Train loss:{:.4f}'
        .format(epoch + 1, epochs, loss))

epoch [1/100], Train loss:0.0238
epoch [2/100], Train loss:0.0072
epoch [3/100], Train loss:0.0046
epoch [4/100], Train loss:0.0043
epoch [5/100], Train loss:0.0042
epoch [6/100], Train loss:0.0041
epoch [7/100], Train loss:0.0038
epoch [8/100], Train loss:0.0035
epoch [9/100], Train loss:0.0035
epoch [10/100], Train loss:0.0034
epoch [11/100], Train loss:0.0033
epoch [12/100], Train loss:0.0032
epoch [13/100], Train loss:0.0031
epoch [14/100], Train loss:0.0031
epoch [15/100], Train loss:0.0030
epoch [16/100], Train loss:0.0027
epoch [17/100], Train loss:0.0027
epoch [18/100], Train loss:0.0027
epoch [19/100], Train loss:0.0026
epoch [20/100], Train loss:0.0026
epoch [21/100], Train loss:0.0025
epoch [22/100], Train loss:0.0024
epoch [23/100], Train loss:0.0023
epoch [24/100], Train loss:0.0023
epoch [25/100], Train loss:0.0023
epoch [26/100], Train loss:0.0023
epoch [27/100], Train loss:0.0023
epoch [28/100], Train loss:0.0022
epoch [29/100], Train loss:0.0022
epoch [30/100], Train l

In [44]:
data = lobs[80000]
print(data)
data= data.reshape(1,-1)

data = minmax_scale.transform(data)

data = model(Variable(torch.from_numpy(data)))
data = minmax_scale.inverse_transform(data.detach().numpy())
print(data)



[125.         125.         125.         118.         125.
   1.           1.           1.           1.           1.
  99.          99.          99.          99.          99.
   1.           1.           1.           1.           1.
 128.         128.         128.         125.         128.
   1.           1.           1.           1.           1.
  88.          51.          68.          78.          94.
   1.           1.           1.           1.           1.
 385.5        385.51666667 385.53333333 385.55       385.56666667]
[[113.09006153 111.68890654 111.41951206 112.08268816 112.35462376
    0.99376467   0.99014479   0.98862873   0.99542708   0.9959733
   96.99039436  97.28525172  96.63131641  97.07149858  96.68620642
    1.02830081   1.00813816   1.0405895    1.0295236    1.0321383
  119.78622725 117.80931134 116.40433701 118.42424204 117.62183531
    0.99802982   1.00329973   0.98852445   0.99684206   0.99851665
   82.35505271  76.96618928  70.20838663  79.28366144  78.46958133
  

In [55]:
outfile = 'autoencoder.pth'
torch.save(model.state_dict(), f'Models/{outfile}')
filehandler = open('Objects/scalar','wb')
pickle.dump(minmax_scale, filehandler)
filehandler.close()
